Instalação do aplicativo que será usado, no caso o qdrant.

In [ ]:
%pip install qdrant-client fastembed 


Conexão para o qdrant cloud usado, com a url e a chave API.

In [ ]:
from qdrant_client import QdrantClient
# connect to Qdrant Cloud
client = QdrantClient(
    url="coloque sua url aqui",
    api_key="coloque sua chave API aqui",
)

Aqui, é criado a coleção na conexão do qdrant cloud criada anteriormente, com o nome "itens" e tamanho máximo de 384.

In [ ]:
from qdrant_client.models import Distance, VectorParams

# create collection
client.create_collection(
    collection_name="items",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

Na primeira parte do trecho do código, é criado o menu de itens de exemplo, no caso um cardápio, com nome, descrição, preço e categorias dos pratos. A segunda parte é o local que essa lista criada é enviada para o qdrant por meio do "cliente.upsert" dos points (a sequência dos pratos, armazenadas e configuradas).

In [ ]:
from qdrant_client.models import PointStruct
from fastembed import TextEmbedding

# load the embedding model
model = TextEmbedding('BAAI/bge-small-en-v1.5')

menu_items = [
    ("Pad Thai with Tofu", "Stir-fried rice noodles with tofu bean sprouts scallions and crushed peanuts in traditional tamarind sauce", "$13.95", "Noodles"),
    ("Grilled Salmon Fillet", "Wild-caught Atlantic salmon grilled with lemon butter and fresh herbs served with seasonal vegetables", "$24.50", "Seafood Entrees"),
    ("Mushroom Risotto", "Creamy arborio rice with mixed mushrooms parmesan truffle oil and fresh thyme", "$16.75", "Vegetarian"),
    ("Bibimbap Bowl", "Korean rice bowl with seasoned vegetables fried egg gochujang sauce and choice of protein", "$14.50", "Korean Bowls"),
    ("Falafel Wrap", "Crispy chickpea fritters with hummus tahini cucumber tomato and pickled vegetables in warm pita", "$11.25", "Mediterranean"),
    ("Shrimp Tacos", "Three soft tacos with grilled shrimp cabbage slaw chipotle aioli and fresh lime", "$13.00", "Tacos"),
    ("Vegetable Curry", "Mixed vegetables in aromatic coconut curry sauce with jasmine rice and naan bread", "$12.95", "Indian Curries"),
    ("Tuna Poke Bowl", "Fresh ahi tuna with avocado edamame cucumber seaweed salad over sushi rice with spicy mayo", "$16.50", "Poke Bowls"),
    ("Margherita Pizza", "Fresh mozzarella san marzano tomatoes basil and extra virgin olive oil on wood-fired crust", "$14.00", "Pizza"),
    ("Chicken Tikka Masala", "Tandoori chicken in creamy tomato sauce with aromatic spices served with basmati rice", "$15.95", "Indian Entrees"),
    ("Greek Salad", "Romaine lettuce tomatoes cucumbers kalamata olives feta cheese red onion with lemon oregano dressing", "$10.50", "Salads"),
    ("Lobster Roll", "Fresh Maine lobster meat with light mayo on toasted buttery roll served with chips", "$22.00", "Seafood Sandwiches"),
    ("Quinoa Buddha Bowl", "Organic quinoa with roasted chickpeas kale sweet potato tahini dressing and hemp seeds", "$13.50", "Healthy Bowls"),
    ("Beef Pho", "Traditional Vietnamese beef noodle soup with rice noodles fresh herbs bean sprouts and lime", "$12.75", "Noodle Soups"),
    ("Eggplant Parmesan", "Breaded eggplant layered with marinara mozzarella and parmesan served with pasta", "$15.25", "Italian Entrees"),
    ("Crab Cakes", "Maryland-style lump crab cakes with remoulade sauce and mixed greens", "$18.50", "Seafood Appetizers"),
    ("Tofu Stir Fry", "Crispy tofu with broccoli bell peppers snap peas in garlic ginger sauce over steamed rice", "$12.50", "Vegetarian Entrees"),
    ("Salmon Sushi Platter", "12 pieces of fresh salmon nigiri and sashimi with wasabi pickled ginger and soy sauce", "$19.95", "Sushi"),
    ("Caprese Sandwich", "Fresh mozzarella tomatoes basil pesto balsamic glaze on ciabatta bread", "$11.75", "Sandwiches"),
    ("Tom Yum Soup", "Spicy and sour Thai soup with shrimp lemongrass galangal mushrooms and kaffir lime leaves", "$11.50", "Soups"),
    ("Lentil Dal", "Red lentils simmered with turmeric cumin coriander served with rice and naan", "$11.95", "Vegan Entrees"),
    ("Fish and Chips", "Beer-battered cod with crispy fries malt vinegar and tartar sauce", "$16.00", "British Classics"),
    ("Veggie Burger", "House-made black bean and quinoa patty with avocado sprouts tomato on brioche bun", "$13.25", "Burgers"),
    ("Miso Ramen", "Rich miso broth with ramen noodles soft-boiled egg bamboo shoots nori and scallions", "$14.50", "Ramen"),
    ("Stuffed Bell Peppers", "Roasted bell peppers filled with rice vegetables herbs and melted cheese", "$13.75", "Vegetarian Entrees"),
    ("Scallop Risotto", "Pan-seared sea scallops over creamy parmesan risotto with white wine and lemon", "$26.50", "Seafood Specials"),
    ("Spring Rolls", "Fresh rice paper rolls with vegetables tofu rice noodles herbs and peanut dipping sauce", "$8.95", "Appetizers"),
    ("Oyster Po Boy", "Fried oysters with lettuce tomato pickles and remoulade on french bread", "$15.50", "Sandwiches"),
    ("Portobello Mushroom Steak", "Grilled portobello cap marinated in balsamic with roasted vegetables and quinoa", "$14.95", "Vegan Entrees"),
    ("Coconut Shrimp", "Jumbo shrimp breaded in shredded coconut served with sweet chili sauce", "$14.25", "Seafood Appetizers")
]

# embedding generator
points = []
embeddings = model.embed([f"{item[0]} {item[1]}" for item in menu_items])
for i, embedding in enumerate(embeddings):
    vector = embedding.tolist()
    point = PointStruct(
        id=i,
        vector=vector,
        payload={
            "item_name": menu_items[i][0],
            "description": menu_items[i][1],
            "price": menu_items[i][2],
            "category": menu_items[i][3],
        }
    )
    points.append(point)

# upsert points to collection
client.upsert(
  collection_name="items",
  points=points,
)

Aqui é onde é feita a query para o modelo usado, no caso a procura para "vegetarian dishes". E é retornado os pontos mais próximos à query na coleção itens, com o payload e no máximo 5 pratos.

In [ ]:
# generate query embedding
query_text = "vegetarian dishes"
query_vector = next(iter(model.embed(query_text)))

# search for similar menu items
results = client.query_points(
    collection_name="items",
    query=query_vector,
    with_payload=True,
    limit=5
)

# print results
for result in results.points:
    print(f"Item: {result.payload.get('item_name', 'N/A')}")
    print(f"Score: {result.score}")
    print(f"Description: {result.payload['description'][:150]}...")
    print(f"Price: {result.payload.get('price', 'N/A')}")
    print("---")

OBS: Nesse código teve erro de execução, 
"""
---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[18], line 2
      1 from qdrant_client.models import PointStruct
----> 2 from fastembed import TextEmbedding
      4 # load the embedding model
      5 model = TextEmbedding('BAAI/bge-small-en-v1.5')

ModuleNotFoundError: No module named 'fastembed'
"""
Aparentemente, a versão que estou usando do python ainda não tem suporte para essa biblioteca. (problema solucionado no código prático).